# Analysis of cell count per organoid
*Author: Felix Romer*<br/>
*Data: 27/02/2024*<br/>
<br/>
The underlying data is spinning disc microscopy data of organoids, stained with DAPI. Because of technical problems (strong bleaching for nuclei in lower half of organoids) only the top half of the organoids were counted. The radius in z as well as the diameter in x and y direction where measured. With that the surface of the ellipsoid describing the organoid will be calculated.<br/>
For some organoid the attempt was made to calculate all cells. This was often not too successful.

## 1. Set up
Imports, data loading, global variables  

In [1]:
# Import needed packages
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns
import numpy as np
from scipy.special import ellipkinc, ellipeinc
from functions import ellipsoid_surface_area

# Load data
file_loc = 'Confocal image metrics data.xlsx'
RawData = pd.read_excel(file_loc)

# global variables
CF_xy = 1       # is already in µm
CF_z  = 1.234

## 2. Process data
* extract experiment conditions
* calculate ellipsoid surface area
* calculate other features of interest

In [44]:
# Peak into raw data
RawData.head(5)

,Name,Slice,Smaller dimaeer,bigger dimaeter,cellnumber,note1,number cells,Note,Area,Perim,Diameter,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14
0,Early life 1Control1 Ctrl 1,45.0,80.0,90.0,200.0,ok,1034,NaN,2.047000e+52,5.241000e+26,1.717000e+26,655.0,812.0,75.33,1.570000e+26
1,Early life 1Control1 Ctrl 2,NaN,NaN,NaN,NaN,NaN,1317,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Early life 1Control1 Ctrl 3,NaN,NaN,NaN,NaN,NaN,254,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Early life 1Control1 Ctrl 4,NaN,NaN,NaN,NaN,NaN,328,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Early life 1Control1 Ctrl 5,NaN,NaN,NaN,NaN,NaN,979,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [47]:
# remove unneded columns
Data = RawData.iloc[:, :8]

# Define a regular expression to extract one and two-digit numbers within text
pattern = r'(?<!\d)(\d{1,2})(?!\d)'

# Extract meta data from Name column
Data['Replication']             = Data['Name'].str.findall(pattern).str[0].astype(int)
Data['Slide_number']            = Data['Name'].str.findall(pattern).str[1].astype(int)
Data['Organoid_number']         = Data['Name'].str.findall(pattern).str[2].astype(int)
Data['Category']                = Data['Name'].apply(lambda x: 'Control' if 'Control' in x else 'HP')
Data['Infection_time']          = Data['Name'].apply(lambda x: 'Early life' if 'Early life' in x else 'Late life')
Data['dz']                      = Data['Slice'] * CF_z

# Calculate surface of ellipsoid
Data['Ellipsoid_surface_area']  = Data.apply(lambda row: ellipsoid_surface_area(row['dz'], row['Smaller dimaeer'], row['bigger dimaeter']), axis=1)
Data['cells_per_area']          = Data['cellnumber'] / Data['Ellipsoid_surface_area']

Data


,Name,Slice,Smaller dimaeer,bigger dimaeter,cellnumber,note1,number cells,Note,Replication,Slide_number,Organoid_number,Category,Infection_time,dz,Ellipsoid_surface_are,cells_per_area
0,Early life 1Control1 Ctrl 1,45.0,80.0,90.0,200.0,ok,1034,NaN,1,1,1,Control,Early life,55.53,70633.672861,0.002832
1,Early life 1Control1 Ctrl 2,NaN,NaN,NaN,NaN,NaN,1317,NaN,1,1,2,Control,Early life,NaN,NaN,NaN
2,Early life 1Control1 Ctrl 3,NaN,NaN,NaN,NaN,NaN,254,NaN,1,1,3,Control,Early life,NaN,NaN,NaN
3,Early life 1Control1 Ctrl 4,NaN,NaN,NaN,NaN,NaN,328,NaN,1,1,4,Control,Early life,NaN,NaN,NaN
4,Early life 1Control1 Ctrl 5,NaN,NaN,NaN,NaN,NaN,979,NaN,1,1,5,Control,Early life,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,Early life 2 Control slide 2 OG 10,NaN,NaN,NaN,NaN,NaN,340,very good,2,2,10,Control,Early life,NaN,NaN,NaN
86,Early life 2 Control slide 2 OG 11,NaN,NaN,NaN,NaN,NaN,33,looks strange,2,2,11,Control,Early life,NaN,NaN,NaN
87,Early life 2 Control slide 2 OG 12,NaN,NaN,NaN,NaN,NaN,20,NaN,2,2,12,Control,Early life,NaN,NaN,NaN
88,Early life 2 Control slide 2 OG 13,NaN,NaN,NaN,NaN,NaN,7,NaN,2,2,13,Control,Early life,NaN,NaN,NaN


## 3. Statistical analysis

## 4. Visualize Data